In [1]:
import sympy as sy
from IPython.display import display
from main import sympy_addons as sya

# from sympy.physics.mechanics import dynamicsymbols

# symbols
t = sy.Symbol("t", real=True, positive=True)  # time
tau = sy.Symbol("tau", real=True, positive=True)
# s = sy.Symbol("s", real=True, positive=True)
g = sy.Symbol("g", real=True)  # gravity

# x = sy.Symbol("x", real=True) # position
# m = sy.Symbol("m", real=True) # mass

# dynamic symbols
# x = sy.physics.mechanics.dynamicsymbols("x")
# m = sy.physics.mechanics.dynamicsymbols("m")

# functions
x = sy.Function("x")(t)
m = sy.Function("m")(t)
F_T = sy.Function("F_T")(t)

# model
model = sy.Eq(m * x.diff(t, 2), F_T - m * g)
display(model)
# model = model.rhs - model.lhs
# display(model)

Eq(m(t)*Derivative(x(t), (t, 2)), -g*m(t) + F_T(t))

Define $m(t)$

In [2]:
b = sy.Symbol("b", real=True, positive=True)  # specific thrust
m_0 = sy.Symbol("m_0", real=True, positive=True)  # initial mass

m2 = m_0 - sy.integrate(b * F_T.subs(t, tau), (tau, 0, t))
display(m2)

-b*Integral(F_T(tau), (tau, 0, t)) + m_0

Substitute $m(t)$ into model

In [3]:
model_2 = model.subs(m, m2)
display(model_2)

Eq((-b*Integral(F_T(tau), (tau, 0, t)) + m_0)*Derivative(x(t), (t, 2)), -g*(-b*Integral(F_T(tau), (tau, 0, t)) + m_0) + F_T(t))

Take the Laplace transform of the model

In [4]:
model_3 = model_2.rhs - model_2.lhs
display(model_3)

-g*(-b*Integral(F_T(tau), (tau, 0, t)) + m_0) - (-b*Integral(F_T(tau), (tau, 0, t)) + m_0)*Derivative(x(t), (t, 2)) + F_T(t)

In [5]:
model_L = sya.L(model_3)
display(model_L)

((b*g*Integral(F_T(tau), (tau, 0, t)) + b*Derivative(x(t), (t, 2))*Integral(F_T(tau), (tau, 0, t)) - g*m_0 - m_0*Derivative(x(t), (t, 2)) + F_T(t))/s,
 0,
 True)